In [ ]:
import pandas as pd
import glob
import os
import re
from pathlib import Path

In [ ]:
disruption_csv_folder = '/projects/disruptsc/disrupt-sc_cambodia_v2/output/Cambodia/'
pattern = 'disruption_*.csv'

In [ ]:
csv_files = glob.glob(os.path.join(disruption_csv_folder, pattern))
print(f"Found {len(csv_files)} disruption CSV files:")
for file in csv_files:
    print(f"  - {os.path.basename(file)}")

In [ ]:
all_data = []

for file in csv_files:
        df = pd.read_csv(file)
        df['filename'] = os.path.basename(file)    
        all_data.append(df)

merged_df = pd.concat(all_data, ignore_index=True)
print(f"\nMerged: {len(merged_df)} total rows")
print(f"Columns: {list(merged_df.columns)}")
    
print(merged_df['simulation_name'].unique())
print(f"Durations: {merged_df['duration'].unique()}")

merged_df = merged_df.sort_values(by=['simulation_name', 'duration', 'mc_repetition'])

In [ ]:
merged_df

In [ ]:
def extract_time_lag(sim_name):
    if sim_name.startswith("cascading_hazards_"):
        match = re.search(r'cascading_hazards_(\d+)d', sim_name)
        if match:
            return int(match.group(1))
    return 0

merged_df['time_lag'] = merged_df['simulation_name'].apply(extract_time_lag)

In [ ]:
cols = list(merged_df.columns)
if 'time_lag' in cols:
    cols.remove('time_lag')
    sim_index = cols.index('simulation_name')
    cols.insert(sim_index + 1, 'time_lag')
    merged_df = merged_df[cols]

In [ ]:
merged_df.to_csv("cambodia_simulations_results.csv")